## Markov reward process

In any system, though, there are desired states to be in and there are other states
that are less desirable. In this section, we will attach rewards to states/transitions, which
gives us a Markov Reward Process (MRP). We then assess the "value" of each state.

![](img/absorbing.png)

In [2]:
import numpy as np

In [3]:
m = 3
m2 = m ** 2
q = np.zeros(m2)
q[m2 // 2] = 1

In [4]:
def get_P(m, p_up, p_down, p_left, p_right):
    m2 = m ** 2
    P = np.zeros((m2, m2))
    ix_map = {i + 1: (i // m, i % m) for i in range(m2)}
    for i in range(m2):
        for j in range(m2):
            r1, c1 = ix_map[i + 1]
            r2, c2 = ix_map[j + 1]
            rdiff = r1 - r2
            cdiff = c1 - c2
            if rdiff == 0:
                if cdiff == 1:
                    P[i, j] = p_left
                elif cdiff == -1:
                    P[i, j] = p_right
                elif cdiff == 0:
                    if r1 == 0:
                        P[i, j] += p_down
                    elif r1 == m - 1:
                        P[i, j] += p_up
                    if c1 == 0:
                        P[i, j] += p_left
                    elif c1 == m - 1:
                        P[i, j] += p_right
            elif rdiff == 1:
                if cdiff == 0:
                    P[i, j] = p_down
            elif rdiff == -1:
                if cdiff == 0:
                    P[i, j] = p_up
    return P

In [5]:
# P with 'crashed state'
P = np.zeros((m2 + 1, m2 + 1))
P[:m2, :m2] = get_P(3, 0.2, 0.3, 0.25, 0.25)
for i in range(m2):
    P[i, m2] = P[i, i]
    P[i, i] = 0
P[m2, m2] = 1

In [9]:
# transition probability matrix with added 'crashed state'
# from state 9 we can only go to state 9 (0 in row 9 from 0-8)
P

array([[0.  , 0.25, 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.55],
       [0.25, 0.  , 0.25, 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.3 ],
       [0.  , 0.25, 0.  , 0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.55],
       [0.3 , 0.  , 0.  , 0.  , 0.25, 0.  , 0.2 , 0.  , 0.  , 0.25],
       [0.  , 0.3 , 0.  , 0.25, 0.  , 0.25, 0.  , 0.2 , 0.  , 0.  ],
       [0.  , 0.  , 0.3 , 0.  , 0.25, 0.  , 0.  , 0.  , 0.2 , 0.25],
       [0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.  , 0.25, 0.  , 0.45],
       [0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.25, 0.  , 0.25, 0.2 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.25, 0.  , 0.45],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

In [6]:
n = 10 ** 5
avg_rewards = np.zeros(m2)
# Assign rewards to transitions
for s in range(9): # for every init state
    for i in range(n): # simulate n episodes
        crashed = False
        s_next = s
        episode_reward = 0
        while not crashed: # simulate until crash and collect rewards
            s_next = np.random.choice(m2 + 1, p=P[s_next, :])
            if s_next < m2: # inside walls
                episode_reward += 1
            else: # went into wall, crash
                crashed = True
        avg_rewards[s] += episode_reward
avg_rewards /= n

In [7]:
np.round(avg_rewards, 2) # average returns

array([1.46, 2.13, 1.47, 2.45, 3.41, 2.44, 1.99, 2.81, 2.  ])

In [7]:
# estimating 4 state from neighbouring states
(1 + 2.45) * 0.25 + (1 + 2.44) * 0.25 + 0.2 * (1+2.81) + 0.3*(1+2.12)

3.4205

### Estimating the state values iteratively

One of the central ideas in RL is to use the value function definition to estimate the value
functions iteratively. To achieve that, we arbitrarily initialize the state values and use its
definition as an update rule. Since we estimate states based on other estimations, this is
a bootstrapping method. We stop when the maximum update to the state value over all
the states is below a set threshold.

![](img/recursive_relationship.png)

In [11]:
def estimate_state_values(P, m2, threshold):
    v = np.zeros(m2 + 1)
    max_change = threshold
    terminal_state = m2
    while max_change >= threshold:
        max_change = 0
        for s in range(m2 + 1): # for every state
            v_new = 0
            for s_next in range(m2 + 1): # for all other states calculate transition
                r = 1 * (s_next != terminal_state)
                v_new += P[s, s_next] * (r + v[s_next])
            max_change = max(max_change, np.abs(v[s] - v_new))
            v[s] = v_new
    return np.round(v, 2)

In [12]:
estimate_state_values(P, m2, 0.005)

array([1.47, 2.12, 1.47, 2.44, 3.42, 2.44, 1.99, 2.82, 1.99, 0.  ])

• We had to iterate over all possible states. This is intractable when the state space is large.
• We used the transition probabilities explicitly. In a realistic system, we don't know what these probabilities are.